In [46]:
# Utilities for object detector.

import numpy as np
import sys
import tensorflow as tf
import os
from threading import Thread
from datetime import datetime
import cv2

#from utils import label_map_util
#from object_detection.utils import label_map_util
from collections import defaultdict
import time
import copy
import matplotlib.pyplot as plt
%matplotlib inline
from imutils.video import VideoStream
from imutils import face_utils
import imutils
from PIL import Image
import math
import pandas as pd
from keras.preprocessing import image


In [47]:
detection_graph = tf.Graph()
#sys.path.append("..")

# score threshold for showing bounding boxes.
score_thresh = 0.25 #0.27

'''
MODEL_NAME = 'hand_inference_graph'
# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'
# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join(MODEL_NAME, 'hand_label_map.pbtxt')'''

NUM_CLASSES = 1

PATH_TO_CKPT = "model_data/frozen_inference_graph.pb"
PATH_TO_LABELS = "model_data/hand_label_map.pbtxt"

'''# load label map
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(
    label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)'''



'# load label map\nlabel_map = label_map_util.load_labelmap(PATH_TO_LABELS)\ncategories = label_map_util.convert_label_map_to_categories(\n    label_map, max_num_classes=NUM_CLASSES, use_display_name=True)\ncategory_index = label_map_util.create_category_index(categories)'

In [48]:
import tensorflow as tf
from keras.models import load_model as lm
model = lm("/home/administrator/HG/VGG_cross_validated.h5", custom_objects = { "tf": tf })
'''gesture_names = {0: 'None',
                 1: 'Fist',
                 2: 'L',
                 3: 'Okay',
                 4: 'Palm',
                 5: 'Peace'}'''
gesture_names = {0: 'No hand---------------------',
                 1: 'Fist',
                 2: 'L- Shape',
                 3: 'Okay',
                 4: 'Hand',
                 5: 'Victory'}


# parameters
cap_region_x_begin = 0.5  # start point/total width
cap_region_y_end = 0.8  # start point/total width
threshold = 60  # binary threshold
blurValue = 41  # GaussianBlur parameter
bgSubThreshold = 50
learningRate = 0
# General Settings
prediction = ''
action = ''
score = 0
img_counter = 500
isBgCaptured = 0

bgModel = cv2.createBackgroundSubtractorMOG2(0, bgSubThreshold)

def predict_image(image):
    print("calling")
    image = np.array(image, dtype='float32')
    image /= 255
    pred_array = model.predict(image)

    # model.predict() returns an array of probabilities - 
    # np.argmax grabs the index of the highest probability.
    result = gesture_names[np.argmax(pred_array)]
    
    # A bit of magic here - the score is a float, but I wanted to
    # display just 2 digits beyond the decimal point.
    score = float("%0.2f" % (max(pred_array[0]) * 100))
    print(f'Result: {result}, Score: {score}')
    return result, score


def remove_background(frame):
    fgmask = bgModel.apply(frame, learningRate=learningRate)
    kernel = np.ones((3, 3), np.uint8)
    fgmask = cv2.erode(fgmask, kernel, iterations=1)
    res = cv2.bitwise_and(frame, frame, mask=fgmask)
    return res

def predict_rgb_image_vgg(image):
    image = np.array(image, dtype='float32')
    image /= 255
    pred_array = model.predict(image)
    print(f'pred_array: {pred_array}')
    result = gesture_names[np.argmax(pred_array)]
    print(f'Result: {result}')
    print(max(pred_array[0]))
    score = float("%0.2f" % (max(pred_array[0]) * 100))
    print(result)
    return result, score

In [68]:

# draw the detected bounding boxes on the images
# You can modify this to also draw a label.
def draw_box_on_image(num_hands_detect, score_thresh, scores, boxes, im_width, im_height, image_np):
    print("In side the draw boxes")
    print(num_hands_detect)
    for i in range(int(num_hands_detect[0])):
        if (scores[i] > score_thresh):
            (left, right, top, bottom) = (boxes[i][1] * im_width, boxes[i][3] * im_width,
                                          boxes[i][0] * im_height, boxes[i][2] * im_height) 
            p1 = (int(left), int(top))
            p2 = (int(right), int(bottom))
            cv2.rectangle(image_np, p1, p2, (77, 255, 9), 3, 1)
            Hand_crop_img = image_np[int(top) : int(bottom), int(left) : int(right)]                        
                              
            a = np.array(Hand_crop_img) 
            print(a,"-------------------========")
            if a.size != 0 and scores[i] > 0.8 :
                cv2.imshow("Hand", Hand_crop_img)
                #cv2.imshow("Hand", Hand_crop_img)
                img = remove_background(Hand_crop_img)
                cv2.imshow("Clip Removed BackGround",img)
                # convert the image into binary image
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                blur = cv2.GaussianBlur(gray, (blurValue, blurValue), 0)
                ret, thresh = cv2.threshold(blur, threshold, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
                
                #Pridiction
                target = np.stack((thresh,) * 3, axis=-1)
                target = cv2.resize(target, (224, 224))
                target = target.reshape(1, 224, 224, 3)
                prediction, score = predict_rgb_image_vgg(target)

                # Draw the text
                cv2.putText(image_np, f"Prediction: {prediction} ({score}%)", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1,
                            (255, 0, 0),3)
                #v2.putText(image_np, f"Action: {action}", (50, 80), cv2.FONT_HERSHEY_SIMPLEX, 1,(0, 255, 0),2)  # Draw the text
                #cv2.imshow('original', frame)
                cv2.imshow('ori', thresh)

# Show fps value on image.
def draw_fps_on_image(fps, image_np):
    cv2.putText(image_np, str(fps), (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

In [69]:

# Load a frozen infrerence graph into memory
def load_inference_graph():

    # load frozen tensorflow model into memory
    print("> ====== loading HAND frozen graph into memory")
    detection_graph = tf.Graph()
    with detection_graph.as_default():
        od_graph_def = tf.GraphDef()
        with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
            serialized_graph = fid.read()
            od_graph_def.ParseFromString(serialized_graph)
            tf.import_graph_def(od_graph_def, name='')
        sess = tf.Session(graph=detection_graph)
    print(">  ====== Hand Inference graph loaded.")
    #print (sess)
    #print (detection_graph)
    
    return detection_graph, sess

In [70]:
# Actual detection .. generate scores and bounding boxes given an image
def detect_objects(image_np, detection_graph, sess):
    # Definite input and output Tensors for detection_graph
    print("Detecting objects")
    image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
    # Each box represents a part of the image where a particular object was detected.
    detection_boxes = detection_graph.get_tensor_by_name(
        'detection_boxes:0')
    # Each score represent how level of confidence for each of the objects.
    # Score is shown on the result image, together with the class label.
    detection_scores = detection_graph.get_tensor_by_name(
        'detection_scores:0')
    detection_classes = detection_graph.get_tensor_by_name(
        'detection_classes:0')
    num_detections = detection_graph.get_tensor_by_name(
        'num_detections:0')

    image_np_expanded = np.expand_dims(image_np, axis=0)

    (boxes, scores, classes, num) = sess.run(
        [detection_boxes, detection_scores,
            detection_classes, num_detections],
        feed_dict={image_tensor: image_np_expanded})
    
    #print (boxes, "---------", scores, "-------", classes, "-------", num)
    #print(boxes.shape, "------------------")
    #print(scores.shape, "------------------")
    #print(classes.shape, "------------------")
    #print(num.shape, "------------------")
    #boxes =np.squeeze(boxes)
    #scores = np.squeeze(scores)
    #print(boxes.shape, "------------------")
    #print(scores.shape, "------------------")
    #print(num)
    return num, np.squeeze(boxes), np.squeeze(scores)

In [71]:
cap = cv2.VideoCapture(0) #uses webcam for video
cap.set(5, 30)
print(cap.get(3),cap.get(1))
frameRate = cap.get(5)
print(frameRate)
detection_graph, sess = load_inference_graph()
count=0
while True:
    count+=1             
    frameId = count
    ret, image_np = cap.read()
    t = time.time()
    print("forward = {}".format(time.time() - t))   #Time calculation
    #frameCopy = np.copy(image_np)
    #cv2.imshow('Input', image_np)
    im_width = image_np.shape[1]
    im_height = image_np.shape[0]
    #print(im_width,im_height)
    num, boxes, scores = detect_objects(image_np, detection_graph, sess)
    draw_box_on_image(num, score_thresh, scores, boxes, im_width, im_height, image_np)
    draw_fps_on_image(frameRate, image_np)
    cv2.imshow('OUt', image_np)
    print("Time Taken for frame = {}".format(time.time() - t)) #Time calculation for one frame
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
               
# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()
   

640.0 -1.0
30.0
> ====== loading HAND frozen graph into memory
>  ====== Hand Inference graph loaded.
forward = 9.059906005859375e-06
Detecting objects
In side the draw boxes
[100.]
Time Taken for frame = 1.6351304054260254
forward = 2.1457672119140625e-06
Detecting objects
In side the draw boxes
[100.]
Time Taken for frame = 0.13518929481506348
forward = 3.337860107421875e-06
Detecting objects
In side the draw boxes
[100.]
Time Taken for frame = 0.12640666961669922
forward = 2.86102294921875e-06
Detecting objects
In side the draw boxes
[100.]
Time Taken for frame = 0.06766033172607422
forward = 2.1457672119140625e-06
Detecting objects
In side the draw boxes
[100.]
Time Taken for frame = 0.04921293258666992
forward = 2.86102294921875e-06
Detecting objects
In side the draw boxes
[100.]
[[[ 77 255   9]
  [ 77 255   9]
  [ 77 255   9]
  ...
  [ 77 255   9]
  [ 77 255   9]
  [ 77 255   9]]

 [[ 77 255   9]
  [ 77 255   9]
  [ 77 255   9]
  ...
  [ 77 255   9]
  [ 77 255   9]
  [ 77 255   9

In side the draw boxes
[100.]
[[[ 77 255   9]
  [ 77 255   9]
  [ 77 255   9]
  ...
  [ 77 255   9]
  [ 77 255   9]
  [ 77 255   9]]

 [[ 77 255   9]
  [ 77 255   9]
  [ 77 255   9]
  ...
  [ 77 255   9]
  [ 77 255   9]
  [ 77 255   9]]

 [[ 77 255   9]
  [ 77 255   9]
  [ 77 255   9]
  ...
  [ 77 255   9]
  [ 77 255   9]
  [ 77 255   9]]

 ...

 [[ 77 255   9]
  [ 77 255   9]
  [ 77 255   9]
  ...
  [  4   9   8]
  [ 77 255   9]
  [ 77 255   9]]

 [[ 77 255   9]
  [ 77 255   9]
  [ 77 255   9]
  ...
  [ 77 255   9]
  [ 77 255   9]
  [ 77 255   9]]

 [[ 77 255   9]
  [ 77 255   9]
  [ 77 255   9]
  ...
  [ 77 255   9]
  [ 77 255   9]
  [ 77 255   9]]] -------------------========
Time Taken for frame = 0.07587957382202148
forward = 1.6689300537109375e-06
Detecting objects
In side the draw boxes
[100.]
[[[ 77 255   9]
  [ 77 255   9]
  [ 77 255   9]
  ...
  [ 77 255   9]
  [ 77 255   9]
  [ 77 255   9]]

 [[ 77 255   9]
  [ 77 255   9]
  [ 77 255   9]
  ...
  [ 77 255   9]
  [ 77 255   9

In side the draw boxes
[100.]
Time Taken for frame = 0.07095551490783691
forward = 2.6226043701171875e-06
Detecting objects
In side the draw boxes
[100.]
Time Taken for frame = 0.06562232971191406
forward = 2.384185791015625e-06
Detecting objects
In side the draw boxes
[100.]
Time Taken for frame = 0.05302739143371582
forward = 2.384185791015625e-06
Detecting objects
In side the draw boxes
[100.]
Time Taken for frame = 0.05350852012634277
forward = 2.86102294921875e-06
Detecting objects
In side the draw boxes
[100.]
Time Taken for frame = 0.07718157768249512
forward = 2.86102294921875e-06
Detecting objects
In side the draw boxes
[100.]
Time Taken for frame = 0.056513309478759766
forward = 3.0994415283203125e-06
Detecting objects
In side the draw boxes
[100.]
Time Taken for frame = 0.05877089500427246
forward = 2.384185791015625e-06
Detecting objects
In side the draw boxes
[100.]
Time Taken for frame = 0.06221580505371094
forward = 3.0994415283203125e-06
Detecting objects
In side the dr

In side the draw boxes
[100.]
[[[ 77 255   9]
  [ 77 255   9]
  [ 77 255   9]
  ...
  [ 77 255   9]
  [ 77 255   9]
  [ 77 255   9]]

 [[ 77 255   9]
  [ 77 255   9]
  [ 77 255   9]
  ...
  [ 77 255   9]
  [ 77 255   9]
  [ 77 255   9]]

 [[ 77 255   9]
  [ 77 255   9]
  [ 77 255   9]
  ...
  [ 77 255   9]
  [ 77 255   9]
  [ 77 255   9]]

 ...

 [[ 77 255   9]
  [ 77 255   9]
  [ 77 255   9]
  ...
  [159 161 156]
  [ 77 255   9]
  [ 77 255   9]]

 [[ 77 255   9]
  [ 77 255   9]
  [ 77 255   9]
  ...
  [ 77 255   9]
  [ 77 255   9]
  [ 77 255   9]]

 [[ 77 255   9]
  [ 77 255   9]
  [ 77 255   9]
  ...
  [ 77 255   9]
  [ 77 255   9]
  [ 77 255   9]]] -------------------========
[[[ 77 255   9]
  [ 77 255   9]
  [ 77 255   9]
  ...
  [ 77 255   9]
  [ 77 255   9]
  [ 77 255   9]]

 [[ 77 255   9]
  [ 77 255   9]
  [ 77 255   9]
  ...
  [ 77 255   9]
  [ 77 255   9]
  [ 77 255   9]]

 [[ 77 255   9]
  [ 77 255   9]
  [ 77 255   9]
  ...
  [ 77 255   9]
  [ 77 255   9]
  [ 77 255   9]]



pred_array: [[0.85814804 0.12688246 0.01090848 0.0024949  0.00156615]]
Result: No hand---------------------
0.85814804
No hand---------------------
[[[ 77 255   9]
  [ 77 255   9]
  [ 77 255   9]
  ...
  [ 77 255   9]
  [ 77 255   9]
  [ 77 255   9]]

 [[ 77 255   9]
  [ 77 255   9]
  [ 77 255   9]
  ...
  [ 77 255   9]
  [ 77 255   9]
  [ 77 255   9]]

 [[ 77 255   9]
  [ 77 255   9]
  [ 77 255   9]
  ...
  [ 77 255   9]
  [ 77 255   9]
  [ 77 255   9]]

 ...

 [[ 77 255   9]
  [ 77 255   9]
  [ 77 255   9]
  ...
  [ 17  17  22]
  [ 77 255   9]
  [ 77 255   9]]

 [[ 77 255   9]
  [ 77 255   9]
  [ 77 255   9]
  ...
  [ 77 255   9]
  [ 77 255   9]
  [ 77 255   9]]

 [[ 77 255   9]
  [ 77 255   9]
  [ 77 255   9]
  ...
  [ 77 255   9]
  [ 77 255   9]
  [ 77 255   9]]] -------------------========
Time Taken for frame = 0.3204464912414551
forward = 3.337860107421875e-06
Detecting objects
In side the draw boxes
[100.]
[[[ 77 255   9]
  [ 77 255   9]
  [ 77 255   9]
  ...
  [ 77 255   9]
  

...

pred_array: [[3.4270016e-03 6.4146137e-05 9.9648970e-01 1.3259376e-05 5.8537566e-06]]
Result: L- Shape
0.9964897
L- Shape
[[[ 77 255   9]
  [ 77 255   9]
  [ 77 255   9]
  ...
  [ 77 255   9]
  [ 77 255   9]
  [ 77 255   9]]

 [[ 77 255   9]
  [ 77 255   9]
  [ 77 255   9]
  ...
  [ 77 255   9]
  [ 77 255   9]
  [ 77 255   9]]

 [[ 77 255   9]
  [ 77 255   9]
  [ 77 255   9]
  ...
  [ 77 255   9]
  [ 77 255   9]
  [ 77 255   9]]

 ...

 [[ 77 255   9]
  [ 77 255   9]
  [ 77 255   9]
  ...
  [157 173 166]
  [ 77 255   9]
  [ 77 255   9]]

 [[ 77 255   9]
  [ 77 255   9]
  [ 77 255   9]
  ...
  [ 77 255   9]
  [ 77 255   9]
  [ 77 255   9]]

 [[ 77 255   9]
  [ 77 255   9]
  [ 77 255   9]
  ...
  [ 77 255   9]
  [ 77 255   9]
  [ 77 255   9]]] -------------------========
Time Taken for frame = 0.4998660087585449
forward = 5.9604644775390625e-06
Detecting objects
In side the draw boxes
[100.]
[[[ 77 255   9]
  [ 77 255   9]
  [ 77 255   9]
  ...
  [ 77 255   9]
  [ 77 255   9]
  [ 77 255 

In side the draw boxes
[100.]
Time Taken for frame = 0.07921743392944336
forward = 3.337860107421875e-06
Detecting objects
In side the draw boxes
[100.]
Time Taken for frame = 0.0636138916015625
forward = 2.6226043701171875e-06
Detecting objects
In side the draw boxes
[100.]
Time Taken for frame = 0.06467843055725098
forward = 2.6226043701171875e-06
Detecting objects
In side the draw boxes
[100.]
Time Taken for frame = 0.07898759841918945
forward = 3.5762786865234375e-06
Detecting objects
In side the draw boxes
[100.]
Time Taken for frame = 0.06025528907775879
forward = 2.86102294921875e-06
Detecting objects
In side the draw boxes
[100.]
Time Taken for frame = 0.06336617469787598
forward = 2.384185791015625e-06
Detecting objects
In side the draw boxes
[100.]
Time Taken for frame = 0.06639790534973145
forward = 3.5762786865234375e-06
Detecting objects
In side the draw boxes
[100.]
Time Taken for frame = 0.06833386421203613
forward = 3.0994415283203125e-06
Detecting objects
In side the d

In side the draw boxes
[100.]
Time Taken for frame = 0.07737064361572266
forward = 3.0994415283203125e-06
Detecting objects
In side the draw boxes
[100.]
Time Taken for frame = 0.05946612358093262
forward = 2.6226043701171875e-06
Detecting objects
In side the draw boxes
[100.]
Time Taken for frame = 0.05622529983520508
forward = 2.6226043701171875e-06
Detecting objects
In side the draw boxes
[100.]
Time Taken for frame = 0.056697845458984375
forward = 2.1457672119140625e-06
Detecting objects
In side the draw boxes
[100.]
Time Taken for frame = 0.06946659088134766
forward = 2.86102294921875e-06
Detecting objects
In side the draw boxes
[100.]
Time Taken for frame = 0.05756521224975586
forward = 3.0994415283203125e-06
Detecting objects
In side the draw boxes
[100.]
Time Taken for frame = 0.0636451244354248
forward = 2.1457672119140625e-06
Detecting objects
In side the draw boxes
[100.]
Time Taken for frame = 0.055457353591918945
forward = 2.6226043701171875e-06
Detecting objects
In side t

In side the draw boxes
[100.]
Time Taken for frame = 0.08325672149658203
forward = 2.6226043701171875e-06
Detecting objects
In side the draw boxes
[100.]
Time Taken for frame = 0.05797863006591797
forward = 2.6226043701171875e-06
Detecting objects
In side the draw boxes
[100.]
Time Taken for frame = 0.06415128707885742
forward = 2.6226043701171875e-06
Detecting objects
In side the draw boxes
[100.]
Time Taken for frame = 0.06546401977539062
forward = 3.814697265625e-06
Detecting objects
In side the draw boxes
[100.]
Time Taken for frame = 0.07710909843444824
forward = 3.0994415283203125e-06
Detecting objects
In side the draw boxes
[100.]
Time Taken for frame = 0.06971406936645508
forward = 2.86102294921875e-06
Detecting objects
In side the draw boxes
[100.]
Time Taken for frame = 0.08228111267089844
forward = 2.384185791015625e-06
Detecting objects
In side the draw boxes
[100.]
Time Taken for frame = 0.06275391578674316
forward = 2.384185791015625e-06
Detecting objects
In side the draw

In side the draw boxes
[100.]
Time Taken for frame = 0.07145905494689941
forward = 2.6226043701171875e-06
Detecting objects
In side the draw boxes
[100.]
Time Taken for frame = 0.05507946014404297
forward = 2.384185791015625e-06
Detecting objects
In side the draw boxes
[100.]
Time Taken for frame = 0.06425285339355469
forward = 3.337860107421875e-06
Detecting objects
In side the draw boxes
[100.]
Time Taken for frame = 0.0630037784576416
forward = 2.86102294921875e-06
Detecting objects
In side the draw boxes
[100.]
Time Taken for frame = 0.08167123794555664
forward = 8.58306884765625e-06
Detecting objects
In side the draw boxes
[100.]
Time Taken for frame = 0.06528711318969727
forward = 3.337860107421875e-06
Detecting objects
In side the draw boxes
[100.]
Time Taken for frame = 0.08469009399414062
forward = 3.5762786865234375e-06
Detecting objects
In side the draw boxes
[100.]
Time Taken for frame = 0.06844544410705566
forward = 2.1457672119140625e-06
Detecting objects
In side the draw